In [37]:
from mpetools import get_info_islands #, run_all_functions
import ee
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

%load_ext autoreload
%autoreload 2
import os
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
from datetime import datetime
from coastsat.coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

ModuleNotFoundError: No module named 'statsmodels'

# Choose the island
* Keredhdhoo is a small, almost unihabited island in Huvadhoo Atoll.
* Aslam & Kench (2017): eroding island.

In [2]:
island = 'Keredhdhoo'
country = 'Maldives'

# Retrieve all time series about the island
### Run all functions OR retrieve available information

In [4]:
run = False

if run:
    island_info = run_all_functions.run_all_functions(island, country)

else:
    island_info = get_info_islands.retrieve_info_island(island, country)    


-------------------------------------------------------------------
RETRIEVING ISLAND INFORMATION
Island: Keredhdhoo, Maldives
-------------------------------------------------------------------



ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'

# Retrieve coastal change time series (`CoastSat`)

In [5]:
# region of interest (longitude, latitude)
polygon = [[[73.434316, 0.664876], 
            [73.434316, 0.653023],
            [73.443429, 0.653023],
            [73.443429, 0.664876],
            [73.434316, 0.664876]]] 

# it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = SDS_tools.smallest_rectangle(polygon)

# date range
dates = ['2000-01-01', '2022-12-31']

# satellite missions ['L5','L7','L8','L9','S2']
sat_list = ['L5','L7','L8','L9','S2']

# choose Landsat collection 'C01' or 'C02'
collection = 'C02'

# name of the site
sitename = '_'.join([island, country])

# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data', 'coastsat')

# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath': filepath,
         'landsat_collection': collection}

# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs);

Number of images available between 2000-01-01 and 2022-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 0 images
     L7: 61 images
     L8: 286 images
     L9: 30 images
     S2: 409 images
  Total to download: 786 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L5: 0 images
     L7: 70 images
     L8: 107 images
  Total Tier 2: 177 images


In [6]:
# Retrieve the images
metadata = SDS_download.retrieve_images(inputs)

Number of images available between 2000-01-01 and 2022-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L5: 0 images
     L7: 61 images
     L8: 286 images
     L9: 30 images
     S2: 409 images
  Total to download: 786 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L5: 0 images
     L7: 70 images
     L8: 107 images
  Total Tier 2: 177 images

L5: 0 images

L7: 61 images
100%
L8: 286 images
100%
L9: 30 images
100%
S2: 409 images
100%
Satellite images downloaded from GEE and save in c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Keredhdhoo_Maldives


In [11]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'dist_clouds': 100,         # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3274,       # epsg code of spatial reference system desired for the output
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 1,     # minimum area (in metres^2) for an object to be labelled as a beach
    'min_length_sl': 1,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'latest', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    # add the inputs defined previously
    'inputs': inputs,
}

In [12]:
SDS_preprocess.save_jpg(metadata, settings, use_matplotlib=True)

Saving images as jpg:
L5: 0 images

L7: 61 images
100%
L8: 286 images
100%
L9: 30 images
100%
S2: 409 images
100%
Satellite images saved as .jpg in c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Keredhdhoo_Maldives\jpg_files\preprocessed


In [14]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

Reference shoreline has been saved in c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Keredhdhoo_Maldives


In [15]:
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

Mapping shorelines:

L7:   100%
L8:   100%
L9:   100%
S2:   100%


In [16]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

102 duplicates
15 bad georef


In [17]:
from pyproj import CRS
geomtype = 'points' # choose 'points' or 'lines' for the layer geometry
gdf = SDS_tools.output_to_gdf(output, geomtype)
if gdf is None:
    raise Exception("output does not contain any mapped shorelines")
gdf.crs = CRS(settings['output_epsg']) # set layer projection
# save GEOJSON layer to file
gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'], '%s_output_%s.geojson'%(sitename,geomtype)),
                                driver='GeoJSON', encoding='utf-8')

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend();

In [19]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)
# remove duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_duplicates(output)
# remove inaccurate georeferencing (set threshold to 10 m)
output = SDS_tools.remove_inaccurate_georef(output, 10)

102 duplicates
15 bad georef


In [20]:
%matplotlib qt
transects = SDS_transects.draw_transects(output, settings)

Transect locations saved in c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Keredhdhoo_Maldives


In [21]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
for i,key in enumerate(list(transects.keys())):
    plt.plot(transects[key][0,0],transects[key][0,1], 'bo', ms=5)
    plt.plot(transects[key][:,0],transects[key][:,1],'k-',lw=1)
    plt.text(transects[key][0,0]-100, transects[key][0,1]+100, key,
                va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))


In [22]:
# along-shore distance over which to consider shoreline points to compute the median intersection
settings_transects = {'along_dist':15}
cross_distance = SDS_transects.compute_intersection(output, transects, settings_transects) 

In [24]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),1)
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.plot(output['dates'], cross_distance[key], '-o', ms=6, mfc='w')
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='top', transform=ax.transAxes, fontsize=14)  

In [25]:
# save a .csv file for Excel users
out_dict = dict([])
out_dict['dates'] = output['dates']
for key in transects.keys():
    out_dict[key] = cross_distance[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'transect_time_series.csv')
df.to_csv(fn, sep=',')
print('Time-series of the shoreline change along the transects saved as:\n%s'%fn)

Time-series of the shoreline change along the transects saved as:
c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Keredhdhoo_Maldives\transect_time_series.csv


In [44]:
geojson_file = os.path.join(os.getcwd(),'data','coastsat', '{}_{}'.format(island, country),'{}_{}_transects.geojson'.format(island, country))
transects = SDS_tools.transects_from_geojson(geojson_file)

55 transects have been loaded coordinates are in epsg:4326


In [41]:
output['shorelines'][0].shape

(62, 2)

In [24]:
# load mapped shorelines from 1984
filename_output = os.path.join(os.getcwd(),'data','coastsat', '{}_{}'.format(island, country),'{}_{}_output.pkl'.format(island, country))
with open(filename_output, 'rb') as f:
    output = pickle.load(f) 

# plot the mapped shorelines
fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
plt.title('%d shorelines mapped at Narrabeen from 1984'%len(output['shorelines']))
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
for i,key in enumerate(list(transects.keys())):
    plt.plot(transects[key][0,0],transects[key][0,1], 'bo', ms=5)
    plt.plot(transects[key][:,0],transects[key][:,1],'k-',lw=1)
    plt.text(transects[key][0,0]-100, transects[key][0,1]+100, key,
             va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))


# load long time-series (1984-2021)
filepath = os.path.join(os.getcwd(),'data','coastsat', '{}_{}'.format(island, country),'transect_time_series.csv'.format(island, country))
df = pd.read_csv(filepath, parse_dates=['dates'])
dates = [_.to_pydatetime() for _ in df['dates']]
cross_distance = dict([])
for key in transects.keys():
    cross_distance[key] = np.array(df[key])


In [33]:
df = pd.DataFrame(cross_distance, index=dates)

In [25]:
# remove outliers in the time-series (coastal despiking)
settings_outliers = {'max_cross_change':   40,             # maximum cross-shore change observable between consecutive timesteps
                     'otsu_threshold':     [-.5,0],        # min and max intensity threshold use for contouring the shoreline
                     'plot_fig':           True,           # whether to plot the intermediate steps
                    }
cross_distance = SDS_transects.reject_outliers(cross_distance,output,settings_outliers)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 443 but corresponding boolean dimension is 559

In [34]:
df_m = df.groupby(pd.Grouper(freq='M')).mean()

In [36]:
df_m['1']

2000-03-31 00:00:00+00:00           NaN
2000-04-30 00:00:00+00:00           NaN
2000-05-31 00:00:00+00:00           NaN
2000-06-30 00:00:00+00:00           NaN
2000-07-31 00:00:00+00:00           NaN
                                ...    
2022-08-31 00:00:00+00:00    147.995295
2022-09-30 00:00:00+00:00    150.895970
2022-10-31 00:00:00+00:00    149.025270
2022-11-30 00:00:00+00:00    145.938438
2022-12-31 00:00:00+00:00    150.570715
Freq: M, Name: 1, Length: 274, dtype: float64